In [4]:
from influencer_profiler import InfluencerProfiler, save_model
# from dataloader import create_data_loader
from dataloader_v2 import create_data_loader
from train_loop import train_loop
from transformers import AutoTokenizer, get_linear_schedule_with_warmup

import pandas as pd
from torch import optim
import torch.nn as nn
import torch

In [5]:
MODELS = ['charlieoneill/distilbert-base-uncased-finetuned-tweet_eval-offensive', 
         'pig4431/TweetEval_roBERTa_5E', 
         'tner/twitter-roberta-base-dec2021-tweetner7-random',
         'cardiffnlp/tweet-topic-21-multi',
         'cardiffnlp/twitter-roberta-base-2021-124m'
         ]

# too large 'tner/bertweet-large-tweetner7-all',

EPOCHS = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def translate_class_column(df):
    class_map = {
        "no influencer": 0,
        "nano": 1,
        "micro": 2,
        "macro": 3,
        "mega": 4
    }
    
    if "class" not in df.columns:
        print("Error: DataFrame does not contain a column called 'class'")
        return
    
    df["class"] = df["class"].apply(lambda x: class_map.get(x, x))
    
    return df

df_train = translate_class_column(pd.read_csv('../data/finetune_train_val_test/train.csv'))
df_val = translate_class_column(pd.read_csv('../data/finetune_train_val_test/validate.csv'))
df_test = translate_class_column(pd.read_csv('../data/finetune_train_val_test/test.csv'))

In [6]:
for model_name in MODELS:    
    print(f'Training {model_name}')
    # Retrieve the tokenizer for the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create data loader
    train_data_loader = create_data_loader(df_train, tokenizer, 512, 4)
    val_data_loader = create_data_loader(df_val, tokenizer, 512, 4)
    test_data_loader = create_data_loader(df_test, tokenizer, 512, 4)

    # Create model
    model = InfluencerProfiler(model= model_name, n_classes=5).to(device)
    model.requires_grad_embeddings(True)

    # Set training parameters
    optimizer = optim.AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_data_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
                    optimizer,
                    num_warmup_steps=0,
                    num_training_steps=total_steps
                )

    # Set loss function
    loss_fn = nn.CrossEntropyLoss().to(device)

    # Train model
    model = train_loop(EPOCHS, train_data_loader, val_data_loader, model, loss_fn, optimizer, device, scheduler)

    # Export model
    save_as = model_name.split('/', 1)[1]
    save_model(model, f'{save_as}.pth')

    # Free GPU mem from model
    del(model)
    torch.cuda.empty_cache()
    print()

Training charlieoneill/distilbert-base-uncased-finetuned-tweet_eval-offensive


Some weights of the model checkpoint at charlieoneill/distilbert-base-uncased-finetuned-tweet_eval-offensive were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/3
----------
Train loss 1.607277274131775 accuracy 0.24793388429752067
Val loss   1.6048325598239899 accuracy 0.26666666666666666

Epoch 2/3
----------
Train loss 1.6071053743362427 accuracy 0.2231404958677686
Val loss   1.6030568778514862 accuracy 0.4666666666666667

Epoch 3/3
----------
Train loss 1.6014700166640743 accuracy 0.2892561983471075
Val loss   1.6021323800086975 accuracy 0.4666666666666667

Model saved to distilbert-base-uncased-finetuned-tweet_eval-offensive.pth

